# Homework 2

In this homework, we are going to play with Twitter data.

The data is represented as rows of of [JSON](https://en.wikipedia.org/wiki/JSON#Example) strings.
It consists of [tweets](https://dev.twitter.com/overview/api/tweets), [messages](https://dev.twitter.com/streaming/overview/messages-types), and a small amount of broken data (cannot be parsed as JSON).

For this homework, we will only focus on tweets and ignore all other messages.


## Tweets

A tweet consists of many data fields. [Here is an example](https://gist.github.com/arapat/03d02c9b327e6ff3f6c3c5c602eeaf8b). You can learn all about them in the Twitter API doc. We are going to briefly introduce only the data fields that will be used in this homework.

* `created_at`: Posted time of this tweet (time zone is included)
* `id_str`: Tweet ID - we recommend using `id_str` over using `id` as Tweet IDs, becauase `id` is an integer and may bring some overflow problems.
* `text`: Tweet content
* `user`: A JSON object for information about the author of the tweet
    * `id_str`: User ID
    * `name`: User name (may contain spaces)
    * `screen_name`: User screen name (no spaces)
* `retweeted_status`: A JSON object for information about the retweeted tweet (i.e. this tweet is not original but retweeteed some other tweet)
    * All data fields of a tweet except `retweeted_status`
* `entities`: A JSON object for all entities in this tweet
    * `hashtags`: An array for all the hashtags that are mentioned in this tweet
    * `urls`: An array for all the URLs that are mentioned in this tweet


## Data source

All tweets are collected using the [Twitter Streaming API](https://dev.twitter.com/streaming/overview).


## Users partition

Besides the original tweets, we will provide you with a Pickle file, which contains a partition over 452,743 Twitter users. It contains a Python dictionary `{user_id: partition_id}`. The users are partitioned into 7 groups.

# Part 0: Load data to a RDD

The tweets data is stored on AWS S3. We have in total a little over 1 TB of tweets. We provide 10 MB of tweets for your local development. For the testing and grading on the homework server, we will use different data.

## Testing on the homework server
In the Playground, we provide three different input sizes to test your program: 1 GB, 10 GB, and 100 GB. To test them, read files list from `../Data/hw2-files-1gb.txt`, `../Data/hw2-files-10gb.txt`, `../Data/hw2-files-100gb.txt`, respectively.

For final submission, make sure to read files list from `../Data/hw2-files-final.txt`. Otherwise your program will receive no points

## Local test

For local testing, read files list from `../Data/hw2-files.txt`.
Now let's see how many lines there are in the input files.

1. Make RDD from the list of files in `hw2-files.txt`.
2. Mark the RDD to be cached (so in next operation data will be loaded in memory) 
3. call the `print_count` method to print number of lines in all these files

It should print
```
Number of elements: 2193
```

In [1]:
def print_count(rdd):
    print 'Number of elements:', rdd.count()

In [8]:
%ls /home/arapat/workspace/teaching/cse255-16/BigData-Spark-private/Data/hw2-files.txt

/home/arapat/workspace/teaching/cse255-16/BigData-Spark-private/Data/hw2-files.txt


In [11]:
# Your code here

raw_data = sc.textFile('/home/arapat/workspace/teaching/cse255-16/BigData-Spark-private/Data/hw2-input.txt').cache()

print_count(raw_data)

Number of elements: 2193


# Part 1: Parse JSON strings to JSON objects

Python has built-in support for JSON.

In [12]:
import json

json_example = '''
{
    "id": 1,
    "name": "A green door",
    "price": 12.50,
    "tags": ["home", "green"]
}
'''

json_obj = json.loads(json_example)
json_obj

{u'id': 1,
 u'name': u'A green door',
 u'price': 12.5,
 u'tags': [u'home', u'green']}

## Broken tweets and irrelevant messages

The data of this assignment may contain broken tweets (invalid JSON strings). So make sure that your code is robust for such cases.

In addition, some lines in the input file might not be tweets, but messages that the Twitter server sent to the developer (such as [limit notices](https://dev.twitter.com/streaming/overview/messages-types#limit_notices)). Your program should also ignore these messages.

*Hint:* [Catch the ValueError](http://stackoverflow.com/questions/11294535/verify-if-a-string-is-json-in-python)


(1) Parse raw JSON tweets to obtain valid JSON objects. From all valid tweets, construct a pair RDD of `(user_id, text)`, where `user_id` is the `id_str` data field of the `user` dictionary (read [Tweets](#Tweets) section above), `text` is the `text` data field.

In [13]:
import json

def safe_parse(raw_json):
    try:
        json_obj = json.loads(raw_json) # Not broken
        json_obj["created_at"] # Not a message
        return json_obj
    except:
        return None

In [14]:
tweets = raw_data.map(safe_parse) \
                 .filter(lambda p: p) \
                 .map(lambda p: (p['user']['id_str'], p['text'].encode('utf-8'))) \
                 .cache()

(2) Count the number of different users in all valid tweets (hint: [the `distinct()` method](https://spark.apache.org/docs/latest/programming-guide.html#transformations)).

It should print
```
The number of unique users is: 2083
```

In [15]:
def print_users_count(count):
    print 'The number of unique users is:', count

In [16]:
users = tweets.map(lambda (user, text): user).distinct()
print_users_count(users.count())

The number of unique users is: 2083


# Part 2: Number of posts from each user partition

Load the Pickle file `../Data/users-partition.pickle`, you will get a dictionary which represents a partition over 452,743 Twitter users, `{user_id: partition_id}`. The users are partitioned into 7 groups. For example, if the dictionary is loaded into a variable named `partition`, the partition ID of the user `59458445` is `partition["59458445"]`. These users are partitioned into 7 groups. The partition ID is an integer between 0-6.

Note that the user partition we provide doesn't cover all users appear in the input data.

(1) Load the pickle file.

In [18]:
import pickle

partition = pickle.load(open("../../Data/users-partition.pickle", "rb"))
len(partition)

452743

(2) Count the number of posts from each user partition

Count the number of posts from group 0, 1, ..., 6, plus the number of posts from users who are not in any partition. Assign users who are not in any partition to the group 7.

Put the results of this step into a pair RDD `(group_id, count)` that is sorted by key.

In [19]:
from operator import add

counts = tweets.map(lambda (user, text): (partition.get(user, 7), 1)) \
               .reduceByKey(add) \
               .sortByKey().collect()
print counts

[(0, 81), (1, 199), (2, 45), (3, 313), (4, 86), (5, 221), (6, 400), (7, 798)]


(3) Print the post count using the `print_post_count` function we provided.

It should print

```
Group 0 posted 81 tweets
Group 1 posted 199 tweets
Group 2 posted 45 tweets
Group 3 posted 313 tweets
Group 4 posted 86 tweets
Group 5 posted 221 tweets
Group 6 posted 400 tweets
Group 7 posted 798 tweets
```

In [20]:
def print_post_count(counts):
    for group_id, count in counts:
        print 'Group %d posted %d tweets' % (group_id, count)

In [21]:
print_post_count(counts)

Group 0 posted 81 tweets
Group 1 posted 199 tweets
Group 2 posted 45 tweets
Group 3 posted 313 tweets
Group 4 posted 86 tweets
Group 5 posted 221 tweets
Group 6 posted 400 tweets
Group 7 posted 798 tweets


# Part 3:  Tokens that are relatively popular in each user partition

In this step, we are going to find tokens that are relatively popular in each user partition.

We define the number of mentions of a token $t$ in a specific user partition $k$ as the number of users from the user partition $k$ that ever mentioned the token $t$ in their tweets. Note that even if some users might mention a token $t$ multiple times or in multiple tweets, a user will contribute at most 1 to the counter of the token $t$.

Please make sure that the number of mentions of a token is equal to the number of users who mentioned this token but NOT the number of tweets that mentioned this token.

Let $N_t^k$ be the number of mentions of the token $t$ in the user partition $k$. Let $N_t^{all} = \sum_{i=0}^7 N_t^{i}$ be the number of total mentions of the token $t$.

We define the relative popularity of a token $t$ in a user partition $k$ as the log ratio between $N_t^k$ and $N_t^{all}$, i.e. 

\begin{equation}
p_t^k = \log \frac{C_t^k}{C_t^{all}}.
\end{equation}


You can compute the relative popularity by calling the function `get_rel_popularity`.

(0) Load the tweet tokenizer.

In [1]:
%load happyfuntokenizing.py

Data/                  HW-1.ipynb       HW-2_sol.ipynb  HW-4/
happyfuntokenizing.py  HW-2_sol2.ipynb  HW-3/           'Untitled Document~'


In [24]:
from math import log

tok = Tokenizer(preserve_case=False)

def get_rel_popularity(c_k, c_all):
    return log(1.0 * c_k / c_all) / log(2)


def print_tokens(tokens, gid = None):
    group_name = "overall"
    if gid is not None:
        group_name = "group %d" % gid
    print '=' * 5 + ' ' + group_name + ' ' + '=' * 5
    for t, n in tokens:
        print "%s\t%.4f" % (t, n)
    print

(1) Tokenize the tweets using the tokenizer we provided above named `tok`. Count the number of mentions for each tokens regardless of specific user group.

Call `print_count` function to show how many different tokens we have.

It should print
```
Number of elements: 8979
```

In [25]:
user_token = tweets.flatMap(lambda (user, text):
                                [(user, token) for token in set(tok.tokenize(text))]) \
                   .distinct() \
                   .cache()
overall_tokens = user_token.map(lambda (user, token): (token, 1)) \
                           .reduceByKey(add) \
                           .cache()
print_count(overall_tokens)

Number of elements: 8979


(2) Tokens that are mentioned by too few users are usually not very interesting. So we want to only keep tokens that are mentioned by at least 100 users. Please filter out tokens that don't meet this requirement.

Call `print_count` function to show how many different tokens we have after the filtering.

Call `print_tokens` function to show top 20 most frequent tokens.

In [17]:
freq_tokens = overall_tokens.filter(lambda (t, c): c >= 100).cache()
print_count(freq_tokens)

top20 = freq_tokens.sortBy(lambda (t, c): c, ascending = False) \
                   .take(20)
print_tokens(top20)

Number of elements: 44
===== overall =====
:	1388.0000
rt	1237.0000
.	826.0000
…	673.0000
the	623.0000
trump	582.0000
to	499.0000
,	489.0000
a	404.0000
is	376.0000
in	297.0000
of	292.0000
and	288.0000
for	281.0000
!	269.0000
?	210.0000
on	195.0000
i	192.0000
you	191.0000
this	190.0000



(3) For all tokens that are mentioned by at least 100 users, compute their relative popularity in each user group. Then print the top 10 tokens with highest relative popularity in each user group. In case two tokens have same relative popularity, break the tie by printing the alphabetically smaller one.

**Hint:** Let the relative popularity of a token $t$ be $p$. The order of the items will be satisfied by sorting them using (-p, t) as the key.


In [18]:
freq_tokens_set = set(freq_tokens.map(lambda (t, c): t).collect())
group_token = user_token.filter(lambda (user, token): token in freq_tokens_set) \
                        .map(lambda (user, token): (partition.get(user, 7), token))
group_token.take(10)

[(7, u'trump'),
 (1, u'this'),
 (6, u'\u2026'),
 (0, u'\u2026'),
 (0, u'...'),
 (1, u'a'),
 (6, u':'),
 (6, u'trump'),
 (7, u'of'),
 (7, u'.')]

In [ ]:
tokens = group_token.map(lambda p: (p, 1)) \
                    .reduceByKey(add) \
                    .map(lambda ((group_id, token), count): (token, (group_id, count))) \
                    .groupByKey() \
                    .mapValues(lambda counts: (list(counts), sum(p[1] for p in counts))) \
                    .cache()

In [20]:
rel_popular = tokens.flatMap(lambda (token, (counts, c_all)):
                             [(gid, (token, get_rel_popularity(c, c_all))) for gid, c in counts]) \
                    .cache()
rel_popular.take(10)

[(2, (u'and', -4.710493382805016)),
 (6, (u'and', -2.415037499278844)),
 (0, (u'and', -4.84799690655495)),
 (4, (u'and', -4.46948528330122)),
 (3, (u'and', -2.777607578663552)),
 (7, (u'and', -1.4694852833012202)),
 (1, (u'and', -3.921997487998727)),
 (5, (u'and', -3.040641984497346)),
 (2, (u'clinton', -6.400879436282184)),
 (0, (u'clinton', -4.400879436282184))]

In [ ]:
for k in range(8):
    top10 = rel_popular.filter(lambda p: p[0] == k) \
                       .sortBy(lambda p: (-p[1][1], p[1][0])) \
                       .map(lambda p: p[1]) \
                       .take(10)
    print_tokens(top10, k)

(4) (optional, not for grading) The users partition is generated by a machine learning algorithm that tries to group the users by their political preferences. Three of the user groups are showing supports to Bernie Sanders, Ted Cruz, and Donald Trump. 

If your program looks okay on the local test data, you can try it on the larger input by submitting your program to the homework server. Observe the output of your program to larger input files, can you guess the partition IDs of the three groups mentioned above based on your output?

In [22]:
# Change the values of the following three items to your guesses
users_support = [
    (3, "Bernie Sanders"),
    (5, "Ted Cruz"),
    (6, "Donald Trump")
]

for gid, candidate in users_support:
    print "Users from group %d are most likely to support %s." % (gid, candidate)

Users from group 3 are most likely to support Bernie Sanders.
Users from group 5 are most likely to support Ted Cruz.
Users from group 6 are most likely to support Donald Trump.
